In [2]:
# 導入必要條件
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_ollama import ChatOllama

# 建立 Ollama 模型
model = ChatOllama(
    model="gpt-oss:20b",
    temperatrue=0.7
    )

In [3]:
# 定義主要提示模板 - 餐廳評論分析
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","你是一個專業的餐廳評論家，擅長分析餐廳的優缺點。"),
        ("human","請分析這家餐廳 : {restaurant_info}"),
    ]
)

In [4]:
chain = (
    prompt_template 
    | model 
    | StrOutputParser()
)

In [5]:
restaurant_info = """
這是一家位於台北市中心的義大利餐廳，提供傳統義式料理。
餐廳環境優雅，服務人員親切，但價格偏高，等待時間較長。
食物品質不錯，但份量偏少。適合約會或特殊場合用餐。
"""

result = chain.invoke({"restaurant_info": restaurant_info})

print(result)

**台北市中心的義式風味餐廳—專業評論**

| 項目 | 優點 | 缺點 | 建議/可改進之處 |
|------|------|------|--------------|
| **位置與交通** | 位於台北市中心，交通便利，方便遊客及本地人前往。 | 由於中心區域人潮頻繁，附近可能會有噪音與擁擠。 | 可在門口設置提示音量降低的音效或使用隔音材料，提升用餐靜謐感。 |
| **環境與裝潢** | 餐廳內裝潢典雅，意大利鄉村風格與現代設計相結合，營造出浪漫氛圍。 | 空間相對較小，座位密度較高，對於大團體或高峰時段可能感到擁擠。 | 可設計可調整的桌位或預約制座位安排，避免顧客過度擁擠。 |
| **服務品質** | 服務人員熱情、專業，主動提供餐點建議與酒品配搭。 | 由於人手有限，顧客等待時間長，部分顧客可能感到不耐煩。 | 增加前台接待或預約系統，或設置候位區與飲料服務，減少等待感。 |
| **菜單與口味** | 料理以傳統義大利料理為主，食材新鮮、口味傳統且精緻。 | 份量相對較小，對於想吃得飽的人可能不夠滿足。 | 可推出加餐選項或共享拼盤，滿足不同需求。 |
| **價格** | 價格雖高，但相對於食材品質與服務，屬於合理範圍。 | 高價位使得部分顧客認為性價比略低。 | 可設定不同價位的套餐或季節限定菜單，讓消費者有更多選擇。 |
| **適合場合** | 適合約會、情侶、商務晚餐、特殊場合或紀念日。 | 由於環境較為正式，休閒族群或家庭聚餐可能不太適合。 | 可調整餐廳風格，增設較為休閒的座位區或家庭套餐，擴大客群。 |

### 總結

這家位於台北市中心的義大利餐廳，以其典雅的環境、親切的服務和高品質的傳統料理，在眾多餐廳中脫穎而出。對於尋求浪漫約會或特殊場合用餐的顧客而言，它提供了理想的氛圍和口味體驗。然而，價格較高、等待時間長、份量偏小等因素可能影響部分顧客的滿意度。若能在預約制度、菜單多樣化、座位配置以及價格策略上做適度調整，將進一步提升整體用餐體驗，吸引更廣泛的客群。

**餐廳主題建議**  
- 推出「情侶套餐」或「慶生特別盤」以吸引情侶與慶祝客群。  
- 在旺季或節假日，提供「先預約+快速上菜」服務，以降低等待時間。  
- 可考慮在菜單中加入「分享拼盤」或「加餐選項」，以滿足不同食量需求。  

此評論旨在協助餐廳管理層